# Cookie cutter

The aim of this notebook is to produce summary statistics for Hazards over NCRA regions, with the flexibility to apply the method to any shapefile region.

Typical statistics include median, mean, min, max, 10th, 90th percentiles

This method has used guidance from [https://github.com/aus-ref-clim-data-nci/shapefiles/blob/master/python_tutorial.ipynb]

<div>
<img src="cookie_cutter.jpg" width="500" title="Cookie cutter"/>
</div>

## Step 1 - access needed packages

In [1]:
# navigate to correct working directory

In [2]:
cd /g/data/mn51/users/gt3409/plotting_maps/

/g/data/mn51/users/gt3409/plotting_maps


In [3]:
# import needed packages
from acs_area_statistics import acs_regional_stats, regions

In [4]:
import xarray as xr
import geopandas as gpd
import regionmask
import matplotlib.pyplot as plt
import numpy as np


In [5]:
# read in the shapefile with regions you will use
# from acs_plotting_maps import regions_dict
# ncra_gdf = gpd.read_file(f'/g/data/ia39/aus-ref-clim-data-nci/shapefiles/data/ncra_regions/ncra_regions.shp')
# ncra_gdf["abbrevs"]=['VIC', 'NT','TAS', 'SA', 'NSW', 'WAN', 'WAS', 'SQ', 'NQ']
# regions = regionmask.from_geopandas(ncra_gdf, names="NAME", abbrevs="abbrevs")
regions

<regionmask.Regions 'unnamed'>
overlap:  None

Regions:
0 VIC                Victoria
1  NT      Northern Territory
2 TAS                Tasmania
3  SA         South Australia
4 NSW   New South Wales & ACT
5 WAN Western Australia North
6 WAS Western Australia South
7  SQ        Queensland South
8  NQ        Queensland North

[9 regions]

## Step 2 - Load and prepare data

Open the data you need. 


In [9]:
%%time 
# open Hazard data
# this is a slowish part of the code. It is likely to be reused if you're interested in calculating multiple statistics
filename = "/g/data/ia39/ncra/extratropical_storms/5km/RX1D_AGCD-05i_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1_annual.nc"
ds = xr.open_dataset(filename, use_cftime = True,)

CPU times: user 9.16 ms, sys: 14.9 ms, total: 24.1 ms
Wall time: 132 ms


## Step 3 - Create your mask

There's a range of ways to calculate your mask:
- **regions.mask_3D(ds)** will create a "mask" array that is ```True``` for all grid cells whose centre point falls within a particular state/territory and ```False``` elsewhere
- **regions.mask_3D_frac_approx(ds)** will calculate the fraction of each grid cell that overlaps with each shape
- **mask_10pct** you may create a mask from the fraction values we need to decide on a minimum overlap threshold. eg ```mask_10pct = frac >= 0.1```

Fractional masking```regions.mask_3D_frac_approx(ds)``` which will weigh each lat lon grid by the approximate fraction of the area that lay within the shapefile geometery. It  is probably the most accurate, but more expensive than the simple mask_3D.

The masking methods return an xr.DataArray with three dimensions:  **region**, **lat**, and **lon**.

In [10]:
# create your mask
# you only need one mask. here are two examples. 
# you can also is a keyword fro the function to calculate the mask, but if you're performing multiple calculations, this can be slow.

In [11]:
%%time
# calculate weighted mask this is a very slow part of the code and
# can be reused for any datasets using the same regions and the same lat lon
mask_frac = regions.mask_3D_frac_approx(ds)

CPU times: user 10.8 s, sys: 1.51 s, total: 12.3 s
Wall time: 12.2 s


In [12]:
%%time
mask_centred = regions.mask_3D(ds)

CPU times: user 1.59 s, sys: 80.5 ms, total: 1.67 s
Wall time: 1.67 s


## Step 4: Use function
To apply mask calculate a statistic to summarise each region 

In [17]:
# calculate the stats using the acs_region_fractional_stats function

start, end = ("1991", "2010") 
dims = ("time", "lat", "lon")
how = "mean"

In [19]:
%%time
# default regional summary average, min, max
da_mean = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = dims, how = "mean")
da_min  = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = dims, how = "min")
da_max  = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = dims, how = "max")

da_summary = xr.merge([da_mean, da_min, da_max])
da_summary

CPU times: user 2.5 s, sys: 1.23 s, total: 3.73 s
Wall time: 3.73 s


<xarray.Dataset> Size: 1kB
Dimensions:  (region: 9)
Coordinates:
  * region   (region) int64 72B 0 1 2 3 4 5 6 7 8
    abbrevs  (region) <U3 108B 'VIC' 'NT' 'TAS' 'SA' 'NSW' 'WAN' 'WAS' 'SQ' 'NQ'
    names    (region) <U23 828B 'Victoria' ... 'Queensland North'
Data variables:
    pr_mean  (region) float64 72B 48.3 85.39 56.84 41.99 ... 48.42 64.71 90.65
    pr_min   (region) float64 72B 9.719 6.215 11.95 6.339 ... 4.867 3.367 6.198
    pr_max   (region) float64 72B 275.8 1.102e+03 254.2 ... 634.2 1.054e+03

In [20]:
da_summary.to_dataframe()

,abbrevs,names,pr_mean,pr_min,pr_max
region,,,,,
0,VIC,Victoria,48.296070,9.718750,275.750000
1,NT,Northern Territory,85.385599,6.214761,1101.638987
2,TAS,Tasmania,56.840691,11.953125,254.234374
3,SA,South Australia,41.989914,6.338555,430.078125
4,NSW,New South Wales & ACT,57.876824,7.812500,627.268351
5,WAN,Western Australia North,78.228344,4.326878,1007.358816
6,WAS,Western Australia South,48.422522,4.867188,485.460938
7,SQ,Queensland South,64.710860,3.367188,634.156250
8,NQ,Queensland North,90.654526,6.198144,1054.054628


In [ ]:
## extra things

In [ ]:
da_p90 = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = ("time", "lat", "lon"), how = "quantile", quantile=0.9)

In [ ]:
%%time
# regional mean
da_mean = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = ("time", "lat", "lon"), how = "mean")
da_mean

In [ ]:
%%time
# regional mean
da_max = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = ("time", "lat", "lon"), how = "max")
da_max

In [ ]:
xr.merge([da_mean, da_max, da_p90])

In [ ]:
xr.merge([da_mean, da_max, da_p90]).to_dataframe()

In [ ]:
xr.merge([da_mean, da_max, da_p90])[["names", "pr_p90"]]

In [ ]:
%%time
# select one region
da_mean = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = ("time", "lat", "lon"), how = "mean", select_abbr=["VIC"])
da_mean

In [ ]:
%%time
# select one region and get timeseries
da_mean = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = ("lat", "lon"), how = "mean", select_abbr=["VIC"])
da_mean.squeeze()


In [ ]:
da_mean.squeeze().plot()

In [ ]:
%%time
# regional mean
da_mean = acs_regional_stats(ds=ds, var=var, mask = "fractional", start=start, end=end, dims = ("time", "lat", "lon"), how = "mean")
da_mean.to_dataframe()

In [ ]:
%%time
# regional mean
da_mean = acs_regional_stats(ds=ds, var=var, mask = "centred", start=start, end=end, dims = ("time", "lat", "lon"), how = "mean")
da_mean.to_dataframe()

In [ ]:
%%time
# regional mean
da_mean = acs_regional_stats(ds=ds, var=var, mask = "min_overlap", start=start, end=end, dims = ("time", "lat", "lon"), how = "mean", overlap_threshold=0.3)
da_mean.to_dataframe()


In [ ]:
%%time
# regional mean
da_mean = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = ("lat", "lon"), how = "mean")
da_mean.to_dataframe(dim_order=["region", "time"])

In [ ]:
da_mean.sel(region=(regions.abbrevs.index("NSW"))).plot()

In [ ]:
%%time
# regional mean
da_mean = acs_regional_stats(ds=ds, var=var, mask =mask_frac, start=start, end=end, dims = ("lat", "lon"), how = "mean")
da_mean.to_dataframe(dim_order=["region", "time"])


In [ ]:
# calculate the mean rx1day value from the 20 years for the global warming level (GWL)
ds = ds.sel(time = slice(start, end))
da_mean = ds["pr"].mean(dim="time")
da_mean.plot()